In [3]:
import pandas as pd
import re
import numpy as np
pd.set_option("display.max_columns",101)

In [413]:
data = pd.read_csv('data_post_LDA')

In [414]:
data.head()

,Unnamed: 0,additional_info,rb,name,id,summary_title,location,current_job_company,current_job_title,current_job_desc,current_job_duration,prev_job_company,prev_job_title,prev_job_desc,prev_job_duration,total_exp_durations,education1_title,education1_college,education1_duration,education2_title,education2_college,education2_duration,education3_title,education3_college,education3_duration,skils,links,certifications,awards,work_auth,current_job_duration_months,prev_job_duration_months,total_exp_durations_months,work_authp,prev_job_desc_clean,prev_job_title_cats,0,1,2,3,4,5,6,7,8,9,10,11
0,0,"['TECHNICAL SKILLS\xa0', <br/>, '\xa0', <br/>,...",NaN,Tracy-Ruan,c47f7ac095973653?sp=0,Data Scientist Intern,"San Francisco, CA",NaN,Data Scientist Intern,['• Prepared customer behavior datasets for cl...,August 2017 to November 2017,NaN,Web Designer Assistant,['• Maintained university website (HTML ...,September 2015 to January 2016,"['August 2017 to November 2017', 'September 20...",B.S. in Data Science,University of San Francisco,January 2013 to May 2017,NaN,NaN,NaN,NaN,NaN,NaN,"[['JAVASCRIPT (Less than 1 year)'], ['PYTHON (...",['https://github.com/tracyruan007'],[],[],Authorized to work in the US for any employer,3,4,8,1.0,"['maintained', 'university', 'website', 'html'...",Software Engg,0.006410,0.006410,0.006410,0.006410,0.006410,0.006410,0.736292,0.006410,0.006410,0.199605,0.006410,0.006411
1,1,"['TECHNICAL SKILLS:\xa0', <br/>, 'Languages ...",NaN,Sai-Nadimpalli,19e0d35744cc56a6?sp=0,Junior Data Scientist,"Kansas City, MO",['Sprint Corporation'],Junior Data Scientist,['• Developed a Hybrid Recommendation System f...,May 2017 to August 2017,['IBM'],Software Development Engineer Intern,['• Developed Web and Android Applications for...,September 2015 to January 2016,"['May 2017 to August 2017', 'September 2015 to...",Master of Science in Computer Science in Compu...,University of Missouri,May 2018,Technology,K L University,May 2016,NaN,NaN,NaN,"[['C (Less than 1 year)'], ['CSS (Less than 1 ...",[],[],[],NaN,3,4,7,NaN,"['developed', 'web', 'android', 'application',...",Student,0.003968,0.003968,0.003968,0.003968,0.003968,0.003968,0.003968,0.003968,0.003968,0.956348,0.003968,0.003968
2,2,"['TECHNICAL SKILLS\xa0', <br/>, '• Proficien...",NaN,Nick-Shi,accfd33784428f69?sp=0,Data Scientist Intern,"Malibu, CA",['MarketPsych Data'],Data Scientist Intern,['• Manipulated and interpreted insights fro...,February 2018 to Present,NaN,Technical Consultant,['• Manipulated 12 year historical option da...,January 2018 to Present,"['February 2018 to Present', 'January 2018 to ...",Master of Science in Applied Finance,"Pepperdine University, Graziadio School of Bus...",December 2017,Master of Science in Computing,Imperial College London,October 2016,Bachelor of Science in Software Engineering,"The University of Nottingham, School of Comput...",July 2015,"[['PYTHON (Less than 1 year)'], ['SQL (Less th...",['http://www.linkedin.com/in/nick-shi'],[],[],Sponsorship required to work in the US,1,2,12,0.0,"['manipulated', '12', 'year', 'historical', 'o...",Project Manager,0.005208,0.005208,0.005208,0.005208,0.005208,0.005208,0.005209,0.005208,0.005208,0.005209,0.005208,0.942707
3,3,"['TECHNICAL SKILLS\xa0', <br/>, 'Relevant Cour...",NaN,Harsh-Mehta,2a4af24d87cca9cd?sp=0,Data Scientist,"Bloomington, IN",['Indiana University'],Data Scientist,['• Implemented probabilistic character recogn...,November 2017 to November 2017,['University of Mumbai'],Data Analyst,['• Performed analysis of supermarket data by ...,June 2017 to June 2017,"['November 2017 to November 2017', 'June 2017 ...",Master's in Data Science,"Indiana University, School of Informatics",May 2019,Bachelor's in Information Technology,University of Mumbai,June 2017,NaN,NaN,NaN,"[['APACHE HBASE (Less than 1 year)'], ['ARTIFI...","['http://github.com/humehta/Projects.git', 'ht...",[],"['TSEC Hall of Fame', 'TSEC Deparment Ambassad...",Authorized to work in the US for any employer,0,0,1,1.0,"['performed', 'a

### Fill NA values

In [137]:
data['current_job_duration']=data.current_job_duration.fillna(0)
data['prev_job_duration']=data.prev_job_duration.fillna(0)
data['total_exp_durations']=data.total_exp_durations.fillna(0)

### Convert dates in duration

In [139]:
months = {'January' : 1,'February' : 2, 'March':3, "April":4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
def convert_dates(date_string):
    if date_string == 0:
        return 0
    try:
        from_, to_ = date_string.split(' to ')
        if len(from_.split(' '))==1:
            from_ = 'July '+from_
        from_month, from_year = from_.split(' ')
        if to_.strip() == 'Present':
            to_ = 'March 2018'

        if len(to_.split(' '))==1:
            to_ = 'July '+to_
        to_month, to_year = to_.split(' ')
        return (int(to_year)-int(from_year))*12 + months[to_month]- months[from_month]
    except:
        return 0

def get_total_exp(exp_str):
    exp_list=[]
    if exp_str ==0:
        return 0
    try:
        exp_list = exp_str.replace('[','').replace(']','').replace("'",'').split(',')
    except:
        return 0
    if exp_list is None:
        return 0
    exp=0
    try:
        for x in exp_list:
            exp = exp + convert_dates(x.strip())
        return exp
    except:
        return exp
    

In [140]:
data['current_job_duration_months']= data.current_job_duration.apply(convert_dates)
data['prev_job_duration_months']= data.prev_job_duration.apply(convert_dates)
data['total_exp_durations_months'] = data.total_exp_durations.apply(get_total_exp)

In [144]:
data.work_auth.unique()

array(['Authorized to work in the US for any employer', nan,
       'Sponsorship required to work in the US'], dtype=object)

#### Work Authorization

In [154]:
def process_work_auth(x):
    if 'Authorized' in str(x):
        return 1
    elif 'Sponsorship' in str(x):
        return 0
    else:
        return None

In [155]:
data['work_authp'] = data.work_auth.apply(process_work_auth)

### LDA for previous work title

In [166]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

In [167]:
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

In [250]:
def clean(doc):
    doc = re.sub('<br/>', '',str(doc))
    doc = re.sub('\\xa0', '', doc)
    doc= re.sub('[^A-Za-z0-9\ ]+', '', doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized.split()

In [251]:
doc_clean= data.prev_job_desc.apply(clean)

In [252]:
import gensim
from gensim import corpora

In [254]:
dictionary = corpora.Dictionary(doc_clean)

In [255]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [256]:
Lda= gensim.models.ldamodel.LdaModel

In [257]:
def get_coherence(model):
    coh=0
    for x in model:
        coh+=x[1]
    return coh/len(model)
#get_coherence(ldamodel.top_topics(texts=doc_clean))

In [258]:
ldamodel = Lda(doc_term_matrix, num_topics=9, id2word = dictionary, passes=100)
this_coh = get_coherence(ldamodel.top_topics(texts=doc_clean))

In [406]:
data.to_csv('data_clean_1.csv',index=False)

In [416]:
def clean_titles(doc):
    doc = re.sub('[^A-Za-z0-9\ ]+', '',str(doc).lower())
    return doc

In [299]:
title_list = []
for i in data.prev_job_title.apply(clean_titles):
    title_list = title_list + i.split()
    
df_keys= pd.DataFrame({'keyword':title_list})

In [415]:
def get_job_cat(doc):
    doc = clean_titles(doc)
    if doc is None:
        return 'Others'
    #Student:
    if 'lab' in doc or 'internship' in doc or 'student' in doc or 'teaching' in doc or 'graduate' in doc or 'summer' in doc or 'research' in doc:
        return 'Student'
    if 'machine' in doc or 'learning' in doc:
        return 'Data Scientist'
    if 'data' in doc and 'scientist' in doc:
        return 'Data Scientist'
    if 'data' in doc and 'researcher' in doc:
        return 'Data Scientist'
    if 'data' in doc and 'research' in doc:
        return 'Data Scientist'
    if 'science' in doc:
        return 'Data Scientist'
    if 'database' in doc or 'sql' in doc:
        return 'Data Analyst'
    if 'analyst' in doc or 'analytics' in 'doc' or 'analysis' in doc:
        if 'data' in doc:
            return 'Data Analyst'
        else:
            return 'Business Analyst'
    if 'intern' in doc:
        return 'Student'
    
    if 'postdoctoral' in doc or 'fellow' in doc or 'ph.d' in doc or 'phd' in doc:
        return 'Data Scientist'
    if 'data' in doc:
        return 'Data Analyst'
    if 'marketing' in doc or 'risk' in doc or 'financial' in doc or 'market' in doc:
        return 'Business Analyst'
    if 'business' in doc:
        return 'Business Analyst'
    if 'systems' in doc or 'system' in doc:
        return 'Software Engg' 
    if 'big' in doc or 'hadoop' in doc:
        return 'Data Scientist'
    if 'big' in doc or 'statistical' in doc or 'statistics' in doc or 'statistician' in doc:
        return 'Data Scientist'
    if 'software' in doc or 'developer' in doc or 'development' in doc or 'architect' in doc or 'development' in doc:
        return 'Software Engg'
    if 'engineering' in doc or 'programmer' in doc or 'technology' in doc or 'web' in doc:
        return 'Software Engg'
    if 'consulting' in doc or 'consultant' in doc or 'manager' in doc or 'project' in doc or 'product' in doc or 'management' in doc or 'director' in doc:
        return 'Project Manager'
    else:
        return 'Others'

In [7]:
get_job_cat('Software Development Engineer Intern')

'Student'

In [8]:
data['prev_job_title_cats'] = data.prev_job_title.apply(get_job_cat)

In [417]:
data.current_job_company= data.current_job_company.apply(lambda x: str(x).replace("['",'').replace("']",''))

In [420]:
data['current_job_title_refined']=data.current_job_title.apply(get_job_cat)

In [421]:
data[data.current_job_title_refined=='Data Scientist'].name.count()

5280

In [499]:
dedup_data.current_job_title_refined.value_counts()

Data Scientist      5131
Student             1847
Others              1749
Data Analyst         971
Project Manager      947
Business Analyst     922
Software Engg        707
Name: current_job_title_refined, dtype: int64

In [422]:
dedup_data= data.drop_duplicates(['name','current_job_company', 'current_job_title'])

In [423]:
dedup_data[dedup_data.current_job_title_refined=='Data Scientist'].name.count()

5131

In [424]:
def has_git(x, string='github'):
    if string in str(x).lower():
        return 1
    else: 
        return 0
    
def has_linked(x, string='linked'):
    if string in str(x).lower():
        return 1
    else: 
        return 0

In [425]:
dedup_data['has_github'] = dedup_data.links.apply(has_git)
dedup_data['has_linked'] = dedup_data.links.apply(has_linked)

/Users/deveshmaheshwari/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/deveshmaheshwari/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [426]:
def get_raw_edu_level(edu):
    
    bs = ['BA','B.A.','B. A.', 'BS', 'B.S', 'B.Tech','BTech','Btech','Bachelor','BE','B.E','bachelor', 'B.Sc','BSc']
    ms = ['MS','MA','M. A.','M.Tech','MTech','M.S','M.A','Master', 'master','Graduate', 'M.B.A.', 'MBA','M.Ed','M.Sc','MSc']
    phd = ['Ph.D', 'PhD','phd','Phd','PhD','Doctorate', 'PHD','Doctor','DOCTOR', 'PH.D', 'Ph. D', 'D.Phil']
    
    if any([True if x in edu else False for x in bs]):
        return 'Bachelor'
    if any([True if x in edu else False for x in ms]):
        return 'Masters'
    if any([True if x in edu else False for x in phd]):
        return 'PhD'
    else:
        return 'Other'

def is_univ(univ):
    un =['University','UNIVERSITY']
    if any([True if x in univ else False for x in un]):
        return True
    else:
        return False
    
def get_edu_level(row):
    edu1= str(row['education1_title'])
    edu2 = str(row['education2_title'])
    edu3 = str(row['education3_title'])
    univ1= str(row['education1_college'])
    univ2= str(row['education2_college'])
    univ3= str(row['education3_college'])
    t_edu1= get_raw_edu_level(edu1)
    t_edu2 = get_raw_edu_level(edu2)
    t_edu3 = get_raw_edu_level(edu3)
    
    if t_edu1=='Other':
        if t_edu2=='Masters':
            return t_edu2
        elif t_edu2=='Bachelor' and is_univ(univ1):
            return 'Masters'
        elif t_edu2!='Other':
            return t_edu2
        elif t_edu3!='Other':
            return t_edu3
        
        elif is_univ(univ1) or is_univ(univ2) or is_univ(univ3):
            return 'Bachelor'
        else:
            return t_edu1
    else:
        return t_edu1
        
        
    

In [433]:
def get_edu_cat_field(row):
    edu1= str(row['education1_title'])
    edu2 = str(row['education2_title'])
    edu3 = str(row['education3_title'])
    univ1= str(row['education1_college'])
    univ2= str(row['education2_college'])
    univ3= str(row['education3_college'])
    t_edu1= get_raw_edu_level(edu1)
    t_edu2 = get_raw_edu_level(edu2)
    t_edu3 = get_raw_edu_level(edu3)
    
    if t_edu1=='Other':
        if t_edu2=='Masters':
            return 2, edu2
        elif t_edu2=='Bachelor' and is_univ(univ1):
            return 1, edu1
        elif t_edu2!='Other':
            return 2, edu2
        elif t_edu3!='Other':
            return 3, edu3
        elif is_univ(univ1):
            return 1, edu1
        elif is_univ(univ2):
            return 2, edu2
        elif is_univ(univ3):
            return 3, edu3
        else:
            return 0, ''
    else:
        return 1, edu1
    
def get_edu_field(row):
    _, field= get_edu_cat_field(row)
    if len(field.split(' of '))>1:
        return field.split(' of ')[1]
    elif len(field.split(' in '))>1:
        return field.split(' in ')[1]
    else:
        return field
    

def get_edu_cat(row):
    arts=['Physic','physic','biology','ARTS','chemistry','Law','EDUCATION','Cognitive','Philosophy','Education','Literature', "Economics", 'Biology','Chemistry','Geography','Political','Geophysics','Linguistics','Psychology','Human','Arts','Atmospheric','Accounting','Sociology']
    ds=['Analytics', 'Data Scientist','analytics','Data', 'Machine Learning','rtificial','DATA SCIENCE','Artificial']
    biz = ['MBA','M.B.A','Management','Administration','Policy']
    cs = ['Computer', 'CS', 'C.S', 'Computing','COMPUTER','computer','comput','Computational']
    engg = ['Engineering','engineering', 'Engg', 'Information','Electrical','Mech','mech','Electronics', 'Software', 'Web','information', 'Technology','technology', 'System','Informatics','Bioinformatics','Operations']
    stat = ['Statistic', 'Stats', 'Mathematc', 'Maths','statistic','Mathematics']
    alledu = [str(row['education1_title']),str(row['education2_title']),str(row['education3_title'])]
    which_edu, _ = get_edu_cat_field(row)
    if which_edu ==0:
        return 'Other'
    else:
        edu = alledu[which_edu-1]

        if any([True if x in edu else False for x in arts]):
            return 'Arts'
        if any([True if x in edu else False for x in biz]):
            return 'Biz'
        if any([True if x in edu else False for x in ds]):
            return 'DS'
        if any([True if x in edu else False for x in cs]):
            return 'CS'
        if any([True if x in edu else False for x in engg]):
            return 'Engg'
        if any([True if x in edu else False for x in stat]):
            return 'Stat'
        else:
            return 'Other'
        
    

In [434]:
dedup_data['highest_edu_level']= dedup_data.apply(lambda row: get_edu_level(row), axis=1)

/Users/deveshmaheshwari/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [435]:
dedup_data.highest_edu_level.value_counts()

Masters     5899
Bachelor    3189
PhD         2455
Other        731
Name: highest_edu_level, dtype: int64

In [436]:
dedup_data['highest_edu_field_cat']= dedup_data.apply(lambda row: get_edu_cat(row), axis=1)

/Users/deveshmaheshwari/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [437]:
dedup_data.highest_edu_field_cat.value_counts()

Arts     2238
Other    2208
Engg     1798
CS       1795
Stat     1576
DS       1380
Biz      1279
Name: highest_edu_field_cat, dtype: int64

In [578]:
dedup_data=dedup_data.drop('Unnamed: 0', axis=1)

In [4]:
import requests as re
import json
from tqdm import tqdm_notebook
from ratelimiter import RateLimiter
from bs4 import BeautifulSoup

In [5]:
dedup_data = pd.read_csv('dedupdata_emp.csv')

In [24]:
@RateLimiter(max_calls=1, period=1)
def get_company_info(name):
    url = 'https://craft.co/frontend/api/universal-search?q='+name
    this_content = re.get(url)
    data=json.loads(this_content.content.decode('utf8'))
    if len(data)>0:
        return data[0]['name'], data[0]['payload']['company_type'], data[0]['payload']['employees_count']
    else:
        return '','',0

In [23]:
comp='Imfree, Inc'
comp = comp.replace(',','').replace('Inc.','').replace(' Inc', '')
comps= comp.split(' ')
if len(comps)>1:
    comp= ' '.join(comps[0:2])
print(comp)

Imfree


In [33]:
for i, row in tqdm_notebook(dedup_data.iterrows()):
    comp = dedup_data.iloc[i]['current_job_company']
    try:
        comp = comp.replace(',','').replace('Inc', '').replace('Inc.','')
        comps= comp.split(' ')
        if len(comps)>1:
            comp= ' '.join(comps[0:1])
        #print(comp)
        comp_info = get_company_info(comp.strip())
        dedup_data.loc[i,'data_current_comp_name2']= comp_info[0]
        dedup_data.loc[i,'data_current_comp_type2']= comp_info[1]
        dedup_data.loc[i,'data_current_comp_employee2']= comp_info[2]
    except:
        continue
    #break

A Jupyter Widget

In [478]:
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [598]:
dedup_data.to_csv('dedupdata_emp.csv', index=False)

In [584]:
dedup_data = pd.read_csv('dedupdata.csv')

In [131]:
dedup_data[dedup_data.data_current_comp_namef=='New York Code and Design Academy'].sample(10)

,additional_info,rb,name,id,summary_title,location,current_job_company,current_job_title,current_job_desc,current_job_duration,prev_job_company,prev_job_title,prev_job_desc,prev_job_duration,total_exp_durations,education1_title,education1_college,education1_duration,education2_title,education2_college,education2_duration,education3_title,education3_college,education3_duration,skils,links,certifications,awards,work_auth,current_job_duration_months,prev_job_duration_months,total_exp_durations_months,work_authp,prev_job_desc_clean,prev_job_title_cats,0,1,2,3,4,5,6,7,8,9,10,11,current_job_title_refined,has_github,has_linked,highest_edu_level,highest_edu_field_cat,data_current_comp_name,data_current_comp_type,data_current_comp_employee,data_current_comp_name1,data_current_comp_type1,data_current_comp_employee1,data_current_comp_name2,data_current_comp_type2,data_current_comp_employee2,data_current_comp_namef,data_current_comp_typef,data_current_comp_employeef
3266,"['Technical Skills\xa0', <br/>, 'Programming L...",NaN,Samriddhi-Shakya,3ccf374eef7e98eb?sp=0,Data Science Fellow,"New York, NY",NYC Data Science Academy,Data Science Fellow,['A 12 week immersive program involving hands ...,July 2016 to Present,['Sun IT Solutions'],Java Developer,"['Big Data)\xa0', <br/>, 'Online training on H...",September 2015 to April 2016,"['July 2016 to Present', 'September 2015 to Ap...",Master of Science in Geography,Auburn University,May 2015,Bachelor of Engineering in Geomatics,Kathmandu University,November 2011,NaN,NaN,NaN,"[['Data Analysis'], ['Hadoop'], ['machine lear...",['http://blog.nycdatascience.com/author/sam648...,"[<div class=""data_display""><p class=""certifica...",[],Authorized to work in the US for any employer,20,7,48,1.0,"['big', 'dataxa0', 'br', 'online', 'training',...",Software Engg,0.002688,0.002688,0.002688,0.002688,0.002688,0.002688,0.002688,0.002688,0.970429,0.002688,0.002688,0.002688,Data Scientist,0,1,Masters,Arts,NaN,NaN,0.0,,,0.0,New York Code and Design Academy,private,107.0,New York Code and Design Academy,private,107.0
2355,NaN,NaN,Gwendolyn-Fernandez,de65abc81fb3fa5a?sp=0,Data Scientist,"Woodside, NY",NYC Data S,Data Scientist,['Highly intensive program that takes ...,September 2017 to December 2017,['International Planned Parenthood Federation'],"Evaluation Assistant, Organizational Learning ...",['● Played\u200b \u200ba\u200b \u200bkey\u200b...,June 2016 to September 2017,"['September 2017 to December 2017', 'June 2016...",BA in Women's Studies,Wesleyan University,September 2012 to May 2016,NaN,NaN,NaN,NaN,NaN,NaN,"[['SQL (1 year)'], ['Python (Less than 1 year)...",['http://blog.nycdatascience.com/author/gferna...,[],[],Authorized to work in the US for any employer,3,15,29,1.0,"['playedu200b', 'u200bau200b', 'u200bkeyu200b'...",Data Scientist,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,Data Scientist,1,1,Bachelor,Other,NaN,NaN,0.0,,,0.0,New York Code and Design Academy,private,107.0,New York Code and Design Academy,private,107.0
9259,NaN,NaN,Andre-Toujas,8325071683887fe6?sp=0,Data Scientist,"Denver, CO",NYC Data Science Academy,Data Scientist,['Completed a fully immersive 12-week Data Sci...,June 2017 to September 2017,['EDG Projectos Angola'],Program Manager,['Managed all projects (as many as 20 simultan...,April 2012 to July 2017,"['June 2017 to September 2017', 'April 2012 to...",Bachelor of Science in Mechanical Engineering,Louisiana State University,May 2001 to May 2006,NaN,NaN,NaN,NaN,NaN,NaN,"[['Python'], ['R'], ['Machine Learning'], ['Sp...",['https://blog.nycdatascience.com/author/atouj...,[],[],Authorized to work in the US for any employer,3,63,132,1.0,"['managed', 'project', 'many', '20', 'simultan...",Project Manager,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,Data Scientist,1,1,Bachelor,Engg,NaN,NaN,0.0,,,0.0,New York Code and Design Academy,private,107.0,New York Code and 

In [75]:
dedup_data[pd.isnull(dedup_data.data_current_comp_name2)].current_job_title_refined.value_counts()

Data Scientist      130
Others               65
Student              60
Project Manager      40
Software Engg        23
Business Analyst     17
Data Analyst         16
Name: current_job_title_refined, dtype: int64

In [113]:
dedup_data.sample(10)

,additional_info,rb,name,id,summary_title,location,current_job_company,current_job_title,current_job_desc,current_job_duration,prev_job_company,prev_job_title,prev_job_desc,prev_job_duration,total_exp_durations,education1_title,education1_college,education1_duration,education2_title,education2_college,education2_duration,education3_title,education3_college,education3_duration,skils,links,certifications,awards,work_auth,current_job_duration_months,prev_job_duration_months,total_exp_durations_months,work_authp,prev_job_desc_clean,prev_job_title_cats,0,1,2,3,4,5,6,7,8,9,10,11,current_job_title_refined,has_github,has_linked,highest_edu_level,highest_edu_field_cat,data_current_comp_name,data_current_comp_type,data_current_comp_employee,data_current_comp_name1,data_current_comp_type1,data_current_comp_employee1,data_current_comp_name2,data_current_comp_type2,data_current_comp_employee2,data_current_comp_namef,data_current_comp_typef,data_current_comp_employeef
1523,"['Machine Learning: Gradient boosting, XGBoost...",NaN,Priyadharshini-Mathivanan,f7d611dc67268ae0?sp=0,"Software Associate, NGFMS project","Secaucus, NJ",Honeywell Technology Solutions Lab,"Software Associate, NGFMS project",['Performed Verification and Validation in C++...,January 2010 to December 2010,NaN,NaN,NaN,0,['January 2010 to December 2010'],Master of Science in Business Intelligence and...,Stevens Institute of Technology,May 2017,Bachelor of Engineering in Computer Science in...,Anna University,April 2009,NaN,NaN,NaN,"[['python (1 year)'], ['R (1 year)'], ['SAS (1...",['https://1drv.ms/f/s!AnTjtrJBT5pDjxuS48wRUa0-...,[],[],Authorized to work in the US for any employer,11,0,11,1.0,['nan'],Others,0.003623,0.111739,0.003623,0.003623,0.003623,0.003623,0.852028,0.003623,0.003623,0.003623,0.003623,0.003623,Software Engg,1,0,Masters,DS,NaN,NaN,0.0,,,0.0,Honeywell,public,131000.0,Honeywell,public,131000.0
257,NaN,NaN,jinjin-qin,07709420a83429a3?sp=0,Data Scientist Intern,"St. Louis, MO",Scopio,Data Scientist Intern,['1. I designed an image classification system...,May 2016 to September 2016,NaN,NaN,NaN,0,['May 2016 to September 2016'],Master's in Information systems,Washington University in St. Louis,September 2015 to December 2016,Master in Applied Physics,University of Science and Technology of China,September 2009 to July 2012,Bachelor in Applied Physics,University of Science and Technology of China,September 2005 to July 2009,"[['C++ (2 years)'], ['Java (2 years)'], ['Matl...","['https://github.com/zoaldyeck', 'https://www....",[],[],Sponsorship required to work in the US,4,0,4,0.0,['nan'],Others,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,Data Scientist,1,1,Masters,Engg,Scopio,private,8.0,Scopio,private,8.0,Scopio,private,8.0,Scopio,private,8.0
4072,"['Academic Projects\xa0', <br/>, '• Documen...",NaN,Thoufeeq-Ahamed-Kajahussain,24d52697f4185934?sp=0,Software Engineer,"Chicago, IL",Tata Consultancy Services Ltd,Software Engineer,"['Client: American Express\xa0', <br/>, '\xa0'...",June 2012 to July 2015,NaN,NaN,NaN,0,['June 2012 to July 2015'],Master of Science in Management Information Sy...,University of Illinois at Chicago (UIC) - Liau...,December 2016,Bachelor of Technology in Electronics and Comm...,"Model Engineering College, Cochin University o...",April 2012,NaN,NaN,NaN,"[['Asp.Net (3 years)'], ['C# (Less than 1 year...",['https://www.linkedin.com/in/ThoufeeqAhamedK'],"[<div class=""data_display""><p class=""certifica...",[],Sponsorship required to work in the US,37,0,37,0.0,['nan'],Others,0.020833,0.020835,0.020834,0.020834,0.020834,0.020833,0.770827,0.020834,0.020833,0.020833,0.020835,0.020834,Software Engg,0,1,Masters,Biz,NaN,NaN,0.0,Tata Consultancy Services,public,390880.0,Tata Consultancy Services,public,390880.0,Tata Consultancy Services,public,390880.0
6829,NaN,NaN,Gary-Smith,e1dd71d43a54c2bd?sp=0,Project Coordinator,"Raleigh, NC","Sagewell,",Project Coordinator,['• Coordinated four ene

In [118]:
dedup_data.data_current_comp_namef.value_counts()

                                           2469
Spanish Broadcasting System                  97
IBM                                          90
Microsoft                                    81
Department of Better Technology              68
AT&T                                         65
Galvanize                                    62
Metis Solutions                              60
General Electric                             56
Booz Allen Hamilton Holding Corporation      55
Data Sciences International                  52
General Assembly                             51
Cognizant                                    48
Capital One                                  47
New York Code and Design Academy             45
Center for Open Science                      45
Tata Consultancy Services                    43
Accenture                                    41
Indianapolis Power & Light Company           40
Cisco                                        38
National Instruments                    

In [119]:
for i, row in tqdm_notebook(dedup_data.iterrows()):
    c1 = 'data_current_comp_namef'
    if dedup_data.loc[i,c1]=='Spanish Broadcasting System':
        dedup_data.loc[i,'data_current_comp_namef']= 'NYC Data Science Academy'
        dedup_data.loc[i,'data_current_comp_typef']= 'coaching'
        dedup_data.loc[i,'data_current_comp_employeef']= 0.0

A Jupyter Widget

In [132]:
dedup_data.to_csv('data_with_comp.csv', index= False)

In [139]:
dedup_data.location.value_counts()

New York, NY                  598
San Francisco, CA             352
Chicago, IL                   331
Seattle, WA                   271
Austin, TX                    250
San Jose, CA                  220
Los Angeles, CA               201
Houston, TX                   196
Atlanta, GA                   190
Dallas, TX                    176
Jersey City, NJ               155
Boston, MA                    151
Washington, DC                138
Brooklyn, NY                  137
San Diego, CA                 128
Denver, CO                    109
Fremont, CA                    87
Sunnyvale, CA                  85
Philadelphia, PA               84
Bellevue, WA                   81
Irvine, CA                     74
Charlotte, NC                  73
Portland, OR                   72
Raleigh, NC                    72
San Francisco Bay Area, CA     70
Minneapolis, MN                67
Santa Clara, CA                66
Mountain View, CA              65
Pittsburgh, PA                 65
Tampa, FL     

In [ ]:
for i, row in tqdm_notebook(dedup_data.iterrows()):
    c1 = 'data_current_comp_employee'
    c2 = 'data_current_comp_employee1'
    if dedup_data.loc[i,c1]!=0.0:
        dedup_data.loc[i,'data_current_comp_namef']= dedup_data.loc[i,'data_current_comp_name']
        dedup_data.loc[i,'data_current_comp_typef']= dedup_data.loc[i,'data_current_comp_type']
        dedup_data.loc[i,'data_current_comp_employeef']= dedup_data.loc[i,'data_current_comp_employee']
    elif dedup_data.loc[i,c2]!=0.0:
        dedup_data.loc[i,'data_current_comp_namef']= dedup_data.loc[i,'data_current_comp_name1']
        dedup_data.loc[i,'data_current_comp_typef']= dedup_data.loc[i,'data_current_comp_type1']
        dedup_data.loc[i,'data_current_comp_employeef']= dedup_data.loc[i,'data_current_comp_employee1']
    else:
        dedup_data.loc[i,'data_current_comp_namef']= dedup_data.loc[i,'data_current_comp_name2']
        dedup_data.loc[i,'data_current_comp_typef']= dedup_data.loc[i,'data_current_comp_type2']
        dedup_data.loc[i,'data_current_comp_employeef']= dedup_data.loc[i,'data_current_comp_employee2']